# Analyzer

> This class contains the main methods to make a polarization analysis

In [1]:
#| default_exp Analyzer

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
from unidecode import unidecode
import regex as re
from mexican_polarization.Loader import Loader 
from wordcloud import WordCloud
from nltk.corpus import stopwords
import matplotlib.pyplot as plt



In [4]:
#| export
class Analyzer:
    def __init__(self,
                 loader:Loader,
                 ):
        """Constructor of the Analyzer class"""
        if loader.processed_bool == False:
            self.methods_available = False
        else:
            self.methods_available = True
        self.loader = loader

    def moral_words_count(self,
                          message_col:str   #Column with the messages
                          ) -> pd.DataFrame: #DataFrame with extra counting columns
        """Returns a dataframe with the count of virtue and vice words in each message in the message_col column"""
        if self.methods_available == False:
            print('You need to process the data first')
            return None
        vice_d = self.loader.vice_dict
        virtue_d = self.loader.virtue_dict
        df = self.loader.processed

        vice = set(vice_d.keys())
        virtue = set(virtue_d.keys())

        def count_words(text,set_of_words):
            words_in_text = text.lower().split()
            i = 0
            for word in words_in_text:
                if word in set_of_words:
                    i += 1
            return i
        
        df['Vice words count'] = df[message_col].apply(lambda x: count_words(str(x),vice)) #¿POR QUE DEBO HACER EL CASTEO?
        df['Virtue words count'] = df[message_col].apply(lambda x: count_words(str(x),virtue))

        return df
    
        
    def moral_words_wc(self,
                       message_col:str, #Column with the messages
                       media_names:str): #Column with media names
        """Creates a wordcloud with the virtue and vice words in the message_col column"""
        if self.methods_available == False:
            print('You need to process the data first')
            return None
        df = self.loader.processed
        stop_words_es = set(stopwords.words('spanish'))
        morales = set()
        morales.update(self.loader.vice_dict.keys())
        morales.update(self.loader.virtue_dict.keys())

        page_names = df[media_names].unique()


        for name in page_names:
            page_df = df[df[media_names] == name]
            
            # Remove rows with empty messages
            page_df = page_df[page_df[media_names].apply(lambda x: isinstance(x, str))]
            
            # Tokenize the messages and remove stop words
            messages = ' '.join([word for sentence in page_df[message_col] for word in sentence.split() if word.lower() not in stop_words_es])
            
            # Filter only the words that are in the moral words set
            moral_words = ' '.join([word for word in messages.split() if word.lower() in morales])

            # Create wordclouds for general words and moral words
            wordcloud_general = WordCloud(width=800, height=400).generate(messages)
            wordcloud_morales = WordCloud(width=800, height=400).generate(moral_words)

            # Show wordcloud for general words
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud_general, interpolation='bilinear')
            plt.title(f'Most Common Words in {name}')
            plt.axis("off")
            plt.show()

            # Show wordcloud for moral words
            plt.figure(figsize=(10, 5))
            plt.imshow(wordcloud_morales, interpolation='bilinear')
            plt.title(f'Most Common Moral Words in {name}')
            plt.axis("off")
            plt.show()


In [ ]:
show_doc(Analyzer.moral_words_count)

---

### Analyzer.moral_words_count

>      Analyzer.moral_words_count (message_col:str)

Returns a dataframe with the count of virtue and vice words in each message in the message_col column

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| message_col | str | Column with the messages |
| **Returns** | **DataFrame** | **DataFrame with extra counting columns** |

In [ ]:
show_doc(Analyzer.moral_words_wc)

---

### Analyzer.moral_words_wc

>      Analyzer.moral_words_wc (message_col:str, media_names:str)

Creates a wordcloud with the virtue and vice words in the message_col column

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| message_col | str | Column with the messages |
| media_names | str | Column with media names |